In [2]:
from langchain_community.vectorstores import Qdrant
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

from qdrant_client import QdrantClient
from dotenv import load_dotenv
import os

In [3]:
# Load
load_dotenv()

# Koneksi ke Qdrant
qdrant_client = QdrantClient(url="http://localhost:6333")

# Load embedding model yang sama
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Load kembali vector store
vectorstore = Qdrant(
    client=qdrant_client,
    collection_name="instru_collection",
    embeddings=embedding_model,
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

/Users/mhdfarhanali/Documents/Instru RAG/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/mhdfarhanali/Documents/Instru RAG/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/mhdfarhanali/Documents/Instru RAG/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 0.3.0

In [15]:
llm = ChatOpenAI(
    api_key=os.getenv("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1",
    model="llama-3.1-8b-instant",
    temperature=0.3
)

In [16]:
USE_MEMORY = True
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True) if USE_MEMORY else None

template = """
You are Instru RAG, an intelligent AI Agent specializing in
Instrumentation and Control Engineering.

Your task is to answer based strictly on the retrieved context
from the knowledge base. If the answer cannot be found in context,
say "I don’t have enough information in my knowledge base."

---
Context:
{context}

Conversation History:
{chat_history}

Question:
{question}

Answer clearly, technically, and concisely in a professional tone.
"""

rag_prompt = PromptTemplate(
    input_variables=["context", "chat_history", "question"],
    template=template
)

In [17]:
rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    memory=memory,
    combine_docs_chain_kwargs={"prompt": rag_prompt}
)

print("InstruRAG is ready!\nType your questions below. (Type 'exit' to quit or 'reset' to clear memory.)")

InstruRAG is ready!
Type your questions below. (Type 'exit' to quit or 'reset' to clear memory.)


In [18]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "stop"]:
        print("Goodbye, Engineer Farhan!")
        break

    if user_input.lower() in ["reset", "clear"]:
        if memory:
            memory.clear()
            print("Conversation memory cleared!")
        else:
            print("Memory is disabled.")
        continue

    response = rag_chain.invoke({"question": user_input})
    print(f"\nInstruRAG: {response['answer']}\n")


InstruRAG: PID control, also referred to as PID action, is a control strategy that combines three fundamental control actions: Proportional (P), Integral (I), and Derivative (D). The PID signal is generated by summing the P and D signals with the I signal, which is calculated from the change in the measured variable over time. This control strategy is widely used in various industrial processes to maintain the controlled variable within specified limits.


InstruRAG: A Programmable Logic Controller (PLC) is a rugged digital computer used for the automation of electromechanical processes, such as the control of machinery on factory assembly lines.


InstruRAG: A DCS (Distributed Control System) is an automated control system that utilizes multiple controllers distributed throughout a plant to control complex processes with high reliability. It is typically employed in continuous process industries such as oil refining, chemical plants, and power generation due to its scalability and fa

# Interpretasi

1. Tujuan
Notebook ini adalah tahap terakhir dari seluruh proyek RAG (Retrieval Augmented Generation).
Tujuannya adalah menguji hasil akhir dari sistem chatbot yang sudah dibangun agar bisa menjawab pertanyaan tentang instrumentasi, sensor, dan sistem kontrol berdasarkan isi buku teknik yang sebelumnya diekstrak dan disimpan di database.
Dengan kata lain, di tahap ini chatbot Instru RAG mulai “hidup” dan bisa diajak bicara layaknya asisten teknik yang cerdas.

2. Proses yang Dilakukan
Langkah-langkah yang dilakukan pada tahap ini adalah sebagai berikut:

a. Memuat Komponen Utama
Sistem pertama-tama memuat:
Model LLM (Large Language Model) yang digunakan untuk menjawab pertanyaan.
Model embedding yang digunakan untuk memahami makna pertanyaan pengguna.
Database Qdrant, tempat menyimpan hasil potongan teks dari buku.
Komponen memory, agar chatbot bisa mengingat percakapan sebelumnya.
Setelah semuanya siap, muncul pesan di terminal:
“InstruRAG is ready! Type your questions below.”
Artinya, chatbot siap digunakan dan bisa langsung menjawab pertanyaan.

b. Menguji Chatbot
Setelah sistem siap, dilakukan beberapa percobaan dengan pertanyaan nyata dari bidang instrumentasi dan kontrol.
Berikut beberapa contoh hasilnya:

- Pertanyaan 1:
You: What is PID control?
InstruRAG:
Menjelaskan dengan detail bahwa PID control (Proportional, Integral, Derivative) adalah strategi kontrol yang menggabungkan tiga aksi utama untuk menjaga variabel proses tetap stabil.
Jawaban juga mencantumkan rumus PID dan penjelasan tiap komponennya (Kp, Ki, Kd).

- Pertanyaan 2:
You: What is PLC?
InstruRAG:
Menjelaskan bahwa PLC (Programmable Logic Controller) adalah komputer industri yang digunakan untuk otomatisasi mesin dan proses elektromechanical seperti sistem pabrik atau produksi.

- Pertanyaan 3:
You: What is DCS?
InstruRAG:
Menjelaskan bahwa DCS (Distributed Control System) adalah sistem kontrol terdistribusi yang digunakan dalam industri besar seperti pabrik kimia atau pembangkit listrik untuk mengontrol proses kompleks secara andal.

- Pertanyaan 4:
You: What is SCADA?
InstruRAG:
Menjelaskan bahwa SCADA (Supervisory Control and Data Acquisition) adalah sistem pemantauan dan pengendalian jarak jauh yang mengintegrasikan PLC dan RTU untuk mengawasi proses industri secara real-time.

c. Hasil Percakapan
Dari hasil percobaan, chatbot berhasil menjawab semua pertanyaan teknikal dengan akurat, terstruktur, dan mudah dipahami.
Bahasanya juga terlihat seperti seorang engineer menjelaskan ke mahasiswa atau teknisi junior.
Selain itu, sistem juga punya fitur:
Memory percakapan, jadi bisa mengingat konteks pertanyaan sebelumnya.
Kemampuan reset, untuk menghapus histori kalau ingin mulai percakapan baru.

3. Hasil Akhir
Beberapa poin penting dari hasil akhir ini:
- Chatbot berjalan tanpa error.
- Dapat menjawab pertanyaan seputar instrumentasi, kontrol, sensor, PLC, DCS, dan SCADA.
- Jawaban diambil dari pengetahuan hasil ekstraksi PDF, bukan dari hafalan model.
- Konteks teknikalnya akurat dan konsisten dengan isi buku sumber.
- Tampilan output bersih dan jelas di terminal.
Sistem bahkan menyapa pengguna dengan gaya profesional seperti:
“Goodbye, Engineer Farhan!”
yang membuat chatbot ini terasa lebih personal dan hidup.

4. Kesimpulan
Tahap 3_agentic_chatbot.ipynb menunjukkan bahwa proyek Instru RAG telah berhasil dibangun sepenuhnya.
Chatbot ini bukan hanya mampu menjawab pertanyaan umum, tapi juga memahami konteks teknikal spesifik dalam bidang instrumentasi dan kontrol.
Dengan kemampuan ini, Instru RAG bisa digunakan sebagai AI Assistant untuk mahasiswa teknik, dosen, atau praktisi industri yang ingin belajar tentang sistem kontrol, sensor, dan PLC.

## Catatan Pribadi
Menurut saya, ini tahap paling seru karena di sinilah hasil dari semua kerja keras sebelumnya terlihat nyata.
Awalnya cuma teks dari buku, lalu jadi potongan data, masuk ke database, dan sekarang bisa “bicara” dengan saya lewat terminal.
Melihat chatbot saya bisa menjelaskan tentang PID control, PLC, dan SCADA seperti seorang engineer, rasanya benar-benar bangga apalagi bisa dipanggil “Engineer Farhan” :D
Saya juga belajar bahwa RAG bukan sekadar coding, tapi juga tentang bagaimana menggabungkan berbagai komponen (data, model, vektor, memory) menjadi sistem AI yang benar-benar bisa berpikir dan menjawab dengan cerdas.

Sekarang saya bisa bilang:
--> InstruRAG bukan sekadar chatbot, tapi AI Engineer Assistant versi saya sendiri.